# Databot Python API Overview and Exercises
This notebook provides an overview of the key classes and functions in the Databot Python API. Each section will explain the purpose of an API component, followed by example code and student exercises.

## 1. DatabotConfig
The `DatabotConfig` class is used to define what sensors should be enabled on the Databot.

**Common Sensor Flags:**
- `pressure`: atmospheric pressure
- `accl`: raw acceleration (including gravity)
- `Laccl`: linear acceleration (gravity-removed)
- `gyro`: gyroscope rotation
- `magneto`: magnetic field strength
- `ambLight`: ambient light in lux
- `Etemp1` and `Etemp2`: external temperature sensors

**Other Parameters:**
- `refresh`: data update interval (in ms)
- `decimal`: number of decimal places for each value
- `led1`, `led2`, `led3`: LED configuration (use `DatabotLEDConfig`)

In [ ]:
from databot import PyDatabot
PyDatabot.get_databot_address(force_address_read=True)

In [ ]:
# Example: Enable pressure and gyroscope sensors
from databot import DatabotConfig
config = DatabotConfig()
config.pressure = True
config.gyro = True
config.refresh = 500  # Update every 500 ms

## 2. DatabotLEDConfig
Used to turn LEDs on/off and set their color.

**Format:**
`DatabotLEDConfig(state: bool, R: int, G: int, B: int)`
- `state`: True to enable, False to turn off
- `R`, `G`, `B`: Color values from 0 to 255

In [ ]:
# Example: Set LED1 to green
from databot import DatabotLEDConfig
config.led1 = DatabotLEDConfig(True, 0, 255, 0)

## 3. PyDatabot and PyDatabotSaveToFileDataCollector
`PyDatabot` is the base class for reading data.
`PyDatabotSaveToFileDataCollector` extends it and saves data to a file.

You can override the `process_databot_data()` method to customize what happens when data is received.

In [ ]:
# Example: Save 50 readings from the pressure sensor to a file
from databot import PyDatabotSaveToFileDataCollector

filename = 'pressure_log.txt'
collector = PyDatabotSaveToFileDataCollector(config, file_name=filename, number_of_records_to_collect=50)
collector.run()

## Student Exercises
Try completing the following tasks to get more comfortable using the Databot API:

**Exercise 1:** Enable the ambient light sensor and record 20 readings to a file.

In [ ]:
# TODO: Enable ambient light sensor and log data
config.ambLight = True
collector = PyDatabotSaveToFileDataCollector(config, file_name='light_log.txt', number_of_records_to_collect=20)
collector.run()

**Exercise 2:** Write a function that returns an LED color depending on light intensity (bright, medium, dark).

In [ ]:
# TODO: Map ambient light to LED color
def get_led_for_light(lux):
    if lux > 800:
        return DatabotLEDConfig(True, 255, 255, 255)  # White (bright)
    elif lux > 300:
        return DatabotLEDConfig(True, 0, 0, 255)      # Blue (medium)
    else:
        return DatabotLEDConfig(True, 255, 0, 0)      # Red (dark)


**Exercise 3:** Modify a collector class to store both pressure and temperature readings, and calculate estimated water density.

In [ ]:
# TODO: Custom logger with pressure and temperature
class WaterDensityLogger(PyDatabotSaveToFileDataCollector):
    def process_databot_data(self, epoch, data):
        pressure = float(data.get('pressure', 101325))
        temp = float(data.get('external_temp_1', 25))
        density = 1000 - (temp - 4)**2 * 0.2  # very rough estimate
        data['density'] = round(density, 2)
        data['timestamp'] = epoch
        with open(self.file_path, 'a') as f:
            f.write(json.dumps(data) + '\n')

**Exercise 4:** Use the `gyro` sensor to detect if the Databot rotates more than 30 degrees in any direction.

In [ ]:
# TODO: Rotation detection logic
def detect_rotation(gyro_data):
    gx = abs(float(gyro_data.get('gyro_x', 0)))
    gy = abs(float(gyro_data.get('gyro_y', 0)))
    gz = abs(float(gyro_data.get('gyro_z', 0)))
    if gx > 30 or gy > 30 or gz > 30:
        return True
    return False

**Exercise 5:** Explore the `DatabotBLEConfig` class. What UUIDs are used for communication?

In [ ]:
# TODO: Print the BLE service and characteristic UUIDs
from databot import DatabotBLEConfig
cfg = DatabotBLEConfig()
print('Service UUID:', cfg.service_uuid)
print('Read UUID:', cfg.read_uuid)
print('Write UUID:', cfg.write_uuid)